In [1]:
from numpy.linalg import norm
import numpy as np
from scipy import sparse
from matplotlib import pyplot as plt
from scipy import optimize
from scipy.sparse import csr_matrix
from scipy.optimize import minimize
from sklearn.datasets import load_iris
from sklearn.datasets import load_svmlight_file
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.utils.validation import check_is_fitted, check_array, check_X_y, check_random_state
from sklearn.model_selection import train_test_split
from LMNN import LargeMarginNearestNeighbor
import time

# Metrics Learning 

**Group Members: Oskar Hint , Xiaoshen Hou,  Kasper Schønberg,  Valentine Van Der Nya**

***
## 0.  Experinment 0 - Iris data 


In [7]:
# data
dataset = load_iris()
X, y = dataset.data, dataset.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, random_state=42)

In [8]:
K=[3,4,5,6,7,8,9,10]
for k in K:
    print "########  K = ",k, " ###########"
    time1 = time.time()
    lmnn = LargeMarginNearestNeighbor(n_neighbors=k)
    lmnn.fit(X_train,y_train)
    time2 = time.time()
    print "Iris LMNN : ", lmnn.score(X_test,y_test), " Time(s) :", (time2-time1)
   
    neigh = KNeighborsClassifier(n_neighbors=k)
    neigh.fit(X_train,y_train)
    print "Iris KNN : ", neigh.score(X_test,y_test)

########  K =  3  ###########
Iris LMNN :  0.952380952381  Time(s) : 0.245000123978
Iris KNN :  0.971428571429
########  K =  4  ###########
Iris LMNN :  0.980952380952  Time(s) : 0.137000083923
Iris KNN :  0.942857142857
########  K =  5  ###########
Iris LMNN :  0.961904761905  Time(s) : 0.434000015259
Iris KNN :  0.971428571429
########  K =  6  ###########
Iris LMNN :  0.980952380952  Time(s) : 0.575999975204
Iris KNN :  0.952380952381
########  K =  7  ###########
Iris LMNN :  0.961904761905  Time(s) : 0.795000076294
Iris KNN :  0.961904761905
########  K =  8  ###########
Iris LMNN :  0.971428571429  Time(s) : 0.628999948502
Iris KNN :  0.961904761905
########  K =  9  ###########
Iris LMNN :  0.971428571429  Time(s) : 1.01499986649
Iris KNN :  0.952380952381
########  K =  10  ###########
Iris LMNN :  0.961904761905  Time(s) : 1.30199980736
Iris KNN :  0.952380952381


***
## 1.  Experinment 1 - USPS data 

In [14]:
def get_USPS_training_data():
    data = load_svmlight_file('ml_data/usps/usps')
    return data[0], data[1]

def get_USPS_testing_data():
    data = load_svmlight_file('ml_data/usps/usps.t')
    return data[0], data[1]

X_train, y_train = get_USPS_training_data()
X_test, y_test = get_USPS_testing_data()

In [15]:
K=5
lmnn = LargeMarginNearestNeighbor(n_neighbors=K)
lmnn.fit(X_train.todense(),y_train)
print "USPS LMNN : ", lmnn.score(X_test.todense(),y_test)

neigh = KNeighborsClassifier(n_neighbors=K)
neigh.fit(X_train.todense(),y_train)
print "USPS KNN : ", neigh.score(X_test.todense(),y_test)

USPS LMNN :  0.943198804185
USPS KNN :  0.950174389636


***
## 2.  Experinment 2 - MNIST data 

In [ ]:
def get_MNIST_training_data():
    data = load_svmlight_file('ml_data/mnist/mnist')
    return data[0], data[1]

def get_MNIST_testing_data():
    data = load_svmlight_file('ml_data/mnist/mnist.t')
    return data[0], data[1]

X_train, y_train = get_MNIST_training_data()
X_test, y_test = get_MNIST_testing_data()